# 第8章: ニューラルネット

第7章で取り組んだポジネガ分類を題材として、ニューラルネットワークで分類モデルを実装する。なお、この章ではPyTorchやTensorFlow、JAXなどの深層学習フレームワークを活用せよ。

## 70. 単語埋め込みの読み込み

事前学習済み単語埋め込みを活用し、$|V| \times d_\rm{emb}$ の単語埋め込み行列$\pmb{E}$を作成せよ。ここで、$|V|$は単語埋め込みの語彙数、$d_\rm{emb}$は単語埋め込みの次元数である。ただし、単語埋め込み行列の先頭の行ベクトル$\pmb{E}_{0,:}$は、将来的にパディング（`<PAD>`）トークンの埋め込みベクトルとして用いたいので、ゼロベクトルとして予約せよ。ゆえに、$\pmb{E}$の2行目以降に事前学習済み単語埋め込みを読み込むことになる。

もし、Google Newsデータセットの[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)（300万単語・フレーズ、300次元）を全て読み込んだ場合、$|V|=3000001, d_\rm{emb}=300$になるはずである（ただ、300万単語の中には、殆ど用いられない稀な単語も含まれるので、語彙を削減した方がメモリの節約になる）。

また、単語埋め込み行列の構築と同時に、単語埋め込み行列の各行のインデックス番号（トークンID）と、単語（トークン）への双方向の対応付けを保持せよ。

In [1]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 702.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 698.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 8.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0

In [2]:
!pip install numpy

In [1]:
import gensim.downloader as api

In [2]:
import gensim.downloader as api

# Google Newsの学習済み単語ベクトルをダウンロード・ロード
word2vec = api.load('glove-wiki-gigaword-50')


[==================================================] 100.0% 66.0/66.0MB downloaded


In [3]:
word2id = {"<PAD>": 0}
id2word = {0: "<PAD>"} #<PAD>を予約しておく

for i, word in enumerate(word2vec.index_to_key):  # index_to_key は語彙のリスト
    word2id[word] = i + 1  # 0は<PAD>だから+1
    id2word[i + 1] = word

In [4]:
import torch
import torch.nn as nn
import numpy as np

In [5]:
vocab_size = len(word2vec) + 1  # <PAD>のために+1
emb_dim = word2vec.vector_size

embedding_tensor = torch.zeros((vocab_size, emb_dim), dtype=torch.float32)

embedding_tensor[1:] = torch.from_numpy(word2vec.vectors)

In [6]:
embedding_tensor.shape

torch.Size([400001, 50])

## 71. データセットの読み込み

[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されている[Stanford Sentiment Treebank (SST)](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip) をダウンロードし、訓練セット（train.tsv）と開発セット（dev.tsv）のテキストと極性ラベルと読み込み、全てのテキストをトークンID列に変換せよ。このとき、単語埋め込みの語彙でカバーされていない単語は無視し、トークン列に含めないことにせよ。また、テキストの全トークンが単語埋め込みの語彙に含まれておらず、空のトークン列となってしまう事例は、訓練セットおよび開発セットから削除せよ（このため、第7章の実験で得られた正解率と比較できなくなることに注意せよ）。

事例の表現方法は任意でよいが、例えば"contains no wit , only labored gags"がネガティブに分類される事例は、次のような辞書オブジェクトで表現すればよい。

```
{'text': 'contains no wit , only labored gags',
 'label': tensor([0.]),
 'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])}
```

この例では、`text`はテキスト、`label`は分類ラベル（ポジティブなら`tensor([1.])`、ネガティブなら`tensor([0.])`）、`input_ids`はテキストのトークン列をID列で表現している。

In [7]:
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
!unzip SST-2.zip

--2025-05-18 08:49:31--  https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.173.132.6, 18.173.132.13, 18.173.132.81, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.173.132.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7439277 (7.1M) [application/zip]
Saving to: ‘SST-2.zip’

SST-2.zip           100%[===================>]   7.09M  --.-KB/s    in 0.08s   

2025-05-18 08:49:31 (93.3 MB/s) - ‘SST-2.zip’ saved [7439277/7439277]

Archive:  SST-2.zip
   creating: SST-2/
  inflating: SST-2/dev.tsv           
   creating: SST-2/original/
  inflating: SST-2/original/README.txt  
  inflating: SST-2/original/SOStr.txt  
  inflating: SST-2/original/STree.txt  
  inflating: SST-2/original/datasetSentences.txt  
  inflating: SST-2/original/datasetSplit.txt  
  inflating: SST-2/original/dictionary.txt  
  inflating: SST-2/original/original_rt_snippets.txt  
  inflating: SST-2/original/sen

In [8]:
import pandas as pd

# データの読み込み
train_df = pd.read_csv("SST-2/train.tsv", sep='\t', )
dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t', )

train_pos_count = (train_df["label"] == 1).sum()
train_neg_count = (train_df["label"] == 0).sum()
dev_pos_count = (dev_df["label"] == 1).sum()
dev_neg_count = (dev_df["label"] == 0).sum()

print("Train: Positive =", train_pos_count, ", Negative =", train_neg_count)
print("Valid: Positive =", dev_pos_count, ", Negative =", dev_neg_count)
train_df.head()

Train: Positive = 37569 , Negative = 29780
Valid: Positive = 444 , Negative = 428


,sentence,label
0,hide new secretions from the parental units,0
1,"contains no wit , only labored gags",0
2,that loves its characters and communicates som...,1
3,remains utterly satisfied to remain the same t...,0
4,on the worst revenge-of-the-nerds clichés the ...,0


In [9]:
def text_to_ids(text, word2id):
    ids = []
    for word in text.split():
        if word in word2id:
            ids.append(word2id[word])
    return ids

In [10]:
def pre_processing(df, word2id):
  data = []
  for index, row in df.iterrows():
    data_unit = {}
    ids = text_to_ids(row["sentence"], word2id)
    if len(ids) > 0:
      data_unit["text"] = row["sentence"]
      data_unit["label"] = torch.tensor([row["label"]], dtype=torch.float32)
      data_unit["input_ids"] = torch.tensor(ids, dtype=torch.long)
      data.append(data_unit)
  return data


In [11]:
train_data = pre_processing(train_df, word2id)
dev_data = pre_processing(dev_df, word2id)

In [12]:
train_data[:5]

[{'text': 'hide new secretions from the parental units ',
  'label': tensor([0.]),
  'input_ids': tensor([ 5709,    51, 52777,    26,     1, 13055,  1504])},
 {'text': 'contains no wit , only labored gags ',
  'label': tensor([0.]),
  'input_ids': tensor([ 2434,    85, 13026,     2,    92, 26399, 31352])},
 {'text': 'that loves its characters and communicates something rather beautiful about human nature ',
  'label': tensor([1.]),
  'input_ids': tensor([   13,  6742,    48,  2154,     6, 36257,   646,   872,  3367,    60,
            474,  1747])},
 {'text': 'remains utterly satisfied to remain the same throughout ',
  'label': tensor([0.]),
  'input_ids': tensor([  949, 14306,  5457,     5,   945,     1,   216,   984])},
 {'text': 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ',
  'label': tensor([0.]),
  'input_ids': tensor([   14,     1,  1608, 72456,     1, 10364,    95, 36510,    61])}]

## 72. Bag of wordsモデルの構築

単語埋め込みの平均ベクトルでテキストの特徴ベクトルを表現し、重みベクトルとの内積でポジティブ及びネガティブを分類するニューラルネットワーク（ロジスティック回帰モデル）を設計せよ。

In [13]:
def input_ids2mean_feature(input_ids, embedding_tensor):
    selected_vectors = embedding_tensor[input_ids]
    mean_vector = torch.mean(selected_vectors, dim=0)

    return mean_vector

In [14]:
ex_mean = input_ids2mean_feature(train_data[0]["input_ids"], embedding_tensor)
ex_mean.shape

torch.Size([50])

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LogisticRegression(nn.Module): #ロジスティック回帰の設計
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        h1 = self.linear(x)
        h2 = torch.sigmoid(h1)
        return h2

## 73. モデルの学習

問題72で設計したモデルの重みベクトルを訓練セット上で学習せよ。ただし、学習中は単語埋め込み行列の値を固定せよ（単語埋め込み行列のファインチューニングは行わない）。また、学習時に損失値を表示するなど、学習の進捗状況をモニタリングできるようにせよ。

In [16]:
model = LogisticRegression(emb_dim, 1)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for data_unit in train_data:
        # 特徴ベクトルの計算
        feature_vector = input_ids2mean_feature(data_unit["input_ids"], embedding_tensor)

        #modelが予測する
        output = model(feature_vector)

        loss = criterion(output, data_unit["label"])

        # バックプロパゲーションとパラメータ更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_data):.4f}')


Epoch 1/10, Loss: 0.5187
Epoch 2/10, Loss: 0.5050
Epoch 3/10, Loss: 0.5049
Epoch 4/10, Loss: 0.5048
Epoch 5/10, Loss: 0.5048
Epoch 6/10, Loss: 0.5047
Epoch 7/10, Loss: 0.5047
Epoch 8/10, Loss: 0.5046
Epoch 9/10, Loss: 0.5046
Epoch 10/10, Loss: 0.5046


## 74. モデルの評価

問題73で学習したモデルの開発セットにおける正解率を求めよ。

In [17]:
def evaluate_accuracy(model, dev_data, embedding_tensor):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data_unit in dev_data:
            feature_vector = input_ids2mean_feature(data_unit["input_ids"], embedding_tensor)

            output = model(feature_vector)

            prob = torch.sigmoid(output)

            pred = (prob >= 0.5).float()

            correct += (pred == data_unit["label"]).sum().item()
            total += 1

    accuracy = correct / total
    return accuracy



In [18]:
accuracy = evaluate_accuracy(model, dev_data, embedding_tensor)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 50.92%


## 75. パディング

複数の事例が与えられたとき、これらをまとめて一つのテンソル・オブジェクトで表現する関数`collate`を実装せよ。与えられた複数の事例のトークン列の長さが異なるときは、トークン列の長さが最も長いものに揃え、0番のトークンIDでパディングをせよ。さらに、トークン列の長さが長いものから順に、事例を並び替えよ。

例えば、訓練データセットの冒頭の4事例が次のように表されているとき、

```
[{'text': 'hide new secretions from the parental units',
  'label': tensor([0.]),
  'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])},
 {'text': 'contains no wit , only labored gags',
  'label': tensor([0.]),
  'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])},
 {'text': 'that loves its characters and communicates something rather beautiful about human nature',
  'label': tensor([1.]),
  'input_ids': tensor([    4,  5053,    45,  3305, 31647,   348,   904,  2815,    47,  1276,  1964])},
 {'text': 'remains utterly satisfied to remain the same throughout',
  'label': tensor([0.]),
  'input_ids': tensor([  987, 14528,  4941,   873,    12,   208,   898])}]
```

`collate`関数を通した結果は以下のようになることが想定される。

```
{'input_ids': tensor([
    [     4,   5053,     45,   3305,  31647,    348,    904,   2815,     47,   1276,   1964],
    [  5785,     66, 113845,     18,     12,  15095,   1594,      0,      0,      0,      0],
    [   987,  14528,   4941,    873,     12,    208,    898,      0,      0,      0,      0],
    [  3475,     87,  15888,     90,  27695,  42637,      0,      0,      0,      0,      0]]),
 'label': tensor([
    [1.],
    [0.],
    [0.],
    [0.]])}
```


In [23]:
def collate(batch):
    # トークン長が長い順にソート
    batch = sorted(batch, key=lambda x: len(x["input_ids"]), reverse=True)

    max_len = len(batch[0]["input_ids"])  #sortしたあとだから，0番目がmax

    padded_input_ids = []
    labels = []

    for data_unit in batch:
        input_ids = data_unit["input_ids"]
        label = data_unit["label"]

        # パディングする数
        pad_len = max_len - len(input_ids)
        padded = torch.cat([input_ids, torch.zeros(pad_len, dtype=torch.long)])  #pad_lenの数だけ下のinput_idsに0を連結する

        padded_input_ids.append(padded) #この時はshape: (max_len)のテンソルのリスト
        labels.append(label)

    # バッチテンソルに変換
    padded_input_ids = torch.stack(padded_input_ids)  # shape: (batch_size, max_len)　torch.stack使えばリストの中にテンソルが入っている状態のものを一つのテンソルにー
    labels = torch.stack(labels)  # shape: (batch_size, 1)

    return {
        "input_ids": padded_input_ids,
        "label": labels
    }


In [26]:
ex_padding = collate(train_data[:5])
ex_padding

{'input_ids': tensor([[   13,  6742,    48,  2154,     6, 36257,   646,   872,  3367,    60,
            474,  1747],
         [   14,     1,  1608, 72456,     1, 10364,    95, 36510,    61,     0,
              0,     0],
         [  949, 14306,  5457,     5,   945,     1,   216,   984,     0,     0,
              0,     0],
         [ 5709,    51, 52777,    26,     1, 13055,  1504,     0,     0,     0,
              0,     0],
         [ 2434,    85, 13026,     2,    92, 26399, 31352,     0,     0,     0,
              0,     0]]),
 'label': tensor([[1.],
         [0.],
         [0.],
         [0.],
         [0.]])}

## 76. ミニバッチ学習

問題75のパディングの処理を活用して、ミニバッチでモデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

## 77. GPU上での学習

問題76のモデル学習をGPU上で実行せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

## 78. 単語埋め込みのファインチューニング

問題77の学習において、単語埋め込みのパラメータも同時に更新するファインチューニングを導入せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

## 79. アーキテクチャの変更

ニューラルネットワークのアーキテクチャを自由に変更し、モデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。例えば、テキストの特徴ベクトル（単語埋め込みの平均ベクトル）に対して多層のニューラルネットワークを通したり、畳み込みニューラルネットワーク（CNN; Convolutional Neural Network）や再帰型ニューラルネットワーク（RNN; Recurrent Neural Network）などのモデルの学習に挑戦するとよい。